In [2]:
import requests

data = requests.get('http://localhost:8080/users').json()
print(data)

[[6, 'Danny Torrances Finger'], [2, 'Leia Organa'], [3, 'Tony Stark'], [4, 'Ellen Ripley'], [5, 'J. Jonah Jameson'], [7, 'Danny Torrance'], [0, 'John Wick'], [1, 'Mike Wazowski'], [8, 'Mario Bru'], [9, 'Felipe Ferrero'], [10, 'Marc Pomar'], [11, 'Julio Porto'], [12, 'Chiquito de la Calzada'], [13, 'Javi Garzas'], [14, 'Clara']]


### Create User:

In [27]:
new_name = {'name': 'Clarita'}
userid = requests.post("http://localhost:8080/user/create", data=new_name).json()
print(userid)

{'error': 'This name already exists! Try a new one ;)'}


### Create chat:

In [9]:
chatid = requests.post("http://localhost:8080/chat/create").json()
print(chatid)

{'chatid': 10}


### Add Message:

In [40]:
new_message = {'userid': 98, 'message': "Probando"}
requests.post("http://localhost:8080/chat/28/addmessage", data=new_message)

<Response [500]>

### Get messages in chat:

In [5]:
messages = requests.get('http://localhost:8080/chat/0/list').json()
print(messages)

[['Hey Mike, whats up??'], ['Dude!!! 😀️ Did you watch the game last night?'], ['No, had to work. How was it?'], ['Awesome! Boyander scored 3 goals!'], ['No way! 😮️'], ['Way...'], ['I bet people went crazy'], ['For sure. We stayed at KuboLoco until 6AM.'], ['Motherfucker! Hungover? 😵️'], ['Dead, burried and still suffering! \U0001f92e️'], ['Paracetamol is your best friend :P'], ['Boyander is the fucking master!'], ['Lets go to have lunch']]


### Get messages from user:

In [43]:
messages = requests.get('http://localhost:8080/user/0/listmessages').json()
print(messages)

[['Hey Mike, whats up??'], ['No, had to work. How was it?'], ['No way! 😮️'], ['I bet people went crazy'], ['Motherfucker! Hungover? 😵️'], ['Sure, boss.'], ['I do! I just dont feel appreciated in my job'], ['I dunno...'], ['Maybe'], ['Thank you, boss! I feel much better...'], ['Paracetamol is your best friend :P']]


### Sentiment analysis:

In [6]:
import nltk # install nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

sid = SentimentIntensityAnalyzer()

sid.polarity_scores("Happy")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mariobru/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5719}

In [35]:
polarity = 0
subjectivity = 0
for e in messages:

    polarity += TextBlob(*e).sentiment[0]
    subjectivity += TextBlob(*e).sentiment[1]

In [36]:
polarity_mean = polarity/len(messages)
subjectivity_mean = subjectivity/len(messages)

print("The polarity mean is {} and the subjectivity mean is {}".format(polarity_mean,subjectivity_mean))

The polarity mean is 0.04423076923076925 and the subjectivity mean is 0.37478632478632473


In [37]:
def chatSent(idchat):
    id_chat = str(idchat)
    URL = 'http://localhost:8080/chat/{}/list'.format(id_chat)
    messages = requests.get(URL).json()
    polarity = 0
    subjectivity = 0
    for e in messages:
        polarity += TextBlob(*e).sentiment[0]
        subjectivity += TextBlob(*e).sentiment[1]
    polarity_mean = polarity/len(messages)
    subjectivity_mean = subjectivity/len(messages)
    return {'Polarity mean': polarity_mean, 'Subjectivity mean': subjectivity_mean}

In [39]:
chatSent(0)

{'Polarity mean': 0.04423076923076925,
 'Subjectivity mean': 0.37478632478632473}